<a href="https://colab.research.google.com/github/K-sushi/Fibonacci-Flux/blob/main/Numerai_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install dependencies
!pip install -q numerapi pandas pyarrow matplotlib lightgbm scikit-learn cloudpickle==2.2.1 seaborn scipy==1.10.1

# Inline plots
%matplotlib inline

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
jax 0.5.2 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
jaxlib 0.5.1 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
scikit-image 0.25.2 requires scipy>=1.11.4, but you have scipy 1.10.1 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [2]:
!pip install --upgrade numpy

  Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.10.1 requires numpy<1.27.0,>=1.19.5, but you have numpy 2.2.5 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
jax 0.5.2 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
jaxlib 0.5.1 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
scikit-image 0.25.2 requires scipy>=1.11.4, but you have scipy 1.10.1 which is incompatible.


In [3]:
!pip install --upgrade pandas scipy


  Using cached scipy-1.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached scipy-1.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (37.6 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1


In [4]:
!pip install numerapi pandas pyarrow matplotlib lightgbm scikit-learn seaborn scipy

In [5]:
!pip install --upgrade --force-reinstall numpy pandas scipy

  Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached scipy-1.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
Using cached scipy-1.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (37.6 MB)
Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl (229 kB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)

In [3]:
import pandas as pd
import json
from numerapi import NumerAPI

# Set the data version to one of the most recent versions
DATA_VERSION = "v5.0"
MAIN_TARGET = "target_cyrusd_20"
TARGET_CANDIDATES = [
  MAIN_TARGET,
  "target_victor_20",
  "target_xerxes_20",
  "target_teager2b_20"
]
FAVORITE_MODEL = "v5_lgbm_ct_blend"

# Download data
napi = NumerAPI()
napi.download_dataset(f"{DATA_VERSION}/train.parquet")
napi.download_dataset(f"{DATA_VERSION}/features.json")
napi.download_dataset(f"{DATA_VERSION}/validation.parquet")
napi.download_dataset(f"{DATA_VERSION}/live_example_preds.parquet")




# Load data
feature_metadata = json.load(open(f"{DATA_VERSION}/features.json"))
feature_cols = feature_metadata["feature_sets"]["medium"]
target_cols = feature_metadata["targets"]
train = pd.read_parquet(f"{DATA_VERSION}/train.parquet")

# Downsample to every 4th era to reduce memory usage and speedup model training (suggested for Colab free tier)
# Comment out the line below to use all the data (higher memory usage, slower model training, potentially better performance)
train = train[train["era"].isin(train["era"].unique()[::4])]

# Print target columns
train[["era"] + target_cols]

,era,target_agnes_20,target_agnes_60,target_alpha_20,target_alpha_60,target_bravo_20,target_bravo_60,target_caroline_20,target_caroline_60,target_charlie_20,...,target_teager2b_60,target_tyler_20,target_tyler_60,target_victor_20,target_victor_60,target_waldo_20,target_waldo_60,target_xerxes_20,target_xerxes_60,target
id,,,,,,,,,,,,,,,,,,,,,
n0007b5abb0c3a25,0001,0.25,0.00,0.25,0.25,0.00,0.00,0.25,0.00,0.25,...,0.50,0.25,0.25,0.25,0.25,0.25,0.00,0.25,0.00,0.25
n003bba8a98662e4,0001,0.25,0.25,0.25,0.00,0.25,0.00,0.25,0.25,0.25,...,0.50,0.25,0.25,0.25,0.00,0.25,0.25,0.25,0.25,0.25
n003bee128c2fcfc,0001,1.00,1.00,1.00,1.00,0.75,1.00,0.75,0.75,0.75,...,1.00,1.00,0.75,0.75,0.75,0.75,1.00,0.75,0.75,0.75
n0048ac83aff7194,0001,0.25,0.25,0.25,0.25,0.50,0.25,0.50,0.25,0.50,...,0.25,0.25,0.25,0.50,0.25,0.25,0.25,0.25,0.25,0.25
n0055a2401ba6480,0001,0.25,0.50,0.25,0.50,0.25,0.50,0.25,0.50,0.25,...,0.50,0.25,0.50,0.25,0.50,0.25,0.50,0.25,0.50,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nffc2d5e4b79a7ae,0573,0.00,0.25,0.00,0.25,0.00,0.25,0.25,0.50,0.00,...,0.50,0.25,0.50,0.25,0.50,0.00,0.50,0.00,0.25,0.00
nffc9844c1c7a6a9,0573,0.50,0.50,0.25,0.50,0.50,0.50,0.50,0.50,0.50,...,0.75,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.25
nffd79773f4109bb,0573,0.50,0.50,0.75,0.50,0.75,0.50,0.50,0.50,0.75,...,0.75,0.50,0.50,0.50,0.50,0.50,0.75,0.50,0.50,0.50


In [4]:
import os
import pandas as pd
import numpy as np
import json
from google.colab import drive
import gc
import time
from tqdm.auto import tqdm
import warnings

# 警告を抑制
warnings.filterwarnings('ignore')

def setup_numerai_environment(mount_drive=True, drive_path='MyDrive/Numerai',
                             use_api=True, api_key=None, api_secret=None,
                             cache_data=True):
    """
    Numerai環境をGoogle Colab上にセットアップする関数

    Parameters:
    -----------
    mount_drive : bool
        Googleドライブをマウントするかどうか
    drive_path : str
        Googleドライブ上のデータ保存パス
    use_api : bool
        Numerai APIを使用するかどうか
    api_key : str, optional
        Numerai API Key（なくても基本的なダウンロードは可能）
    api_secret : str, optional
        Numerai API Secret（なくても基本的なダウンロードは可能）
    cache_data : bool
        データをキャッシュするかどうか

    Returns:
    --------
    dict
        設定情報と保存パスを含む辞書
    """
    config = {'data_path': '/content'}

    # Googleドライブのマウント
    if mount_drive:
        print("Googleドライブをマウント中...")
        drive.mount('/content/drive')
        drive_data_path = f'/content/drive/{drive_path}'
        os.makedirs(drive_data_path, exist_ok=True)
        config['drive_path'] = drive_data_path
        print(f"ドライブパス: {drive_data_path}")

    # Numerai APIのセットアップ
    if use_api:
        try:
            # numerapiのインストール（未インストールの場合）
            try:
                import numerapi
            except ImportError:
                print("numerapiをインストール中...")
                !pip install -q numerapi
                import numerapi

            # API接続のセットアップ
            if api_key and api_secret:
                napi = numerapi.NumerAPI(api_key, api_secret)
                print("APIキーを使用してNumerai APIに接続しました")
            else:
                napi = numerapi.NumerAPI()
                print("公開APIとしてNumerai APIに接続しました")

            config['napi'] = napi
            current_round = napi.get_current_round()
            print(f"現在のラウンド: {current_round}")
        except Exception as e:
            print(f"APIセットアップエラー: {e}")
            use_api = False

    # キャッシュディレクトリの作成
    if cache_data:
        cache_path = '/content/numerai_cache'
        os.makedirs(cache_path, exist_ok=True)
        config['cache_path'] = cache_path

    return config

def download_numerai_data(config, datasets=None, force_download=False, save_to_drive=True):
    """
    Numeraiデータセットをダウンロードする関数

    Parameters:
    -----------
    config : dict
        セットアップ関数から返される設定情報
    datasets : list, optional
        ダウンロードするデータセットのリスト
    force_download : bool
        既存のファイルを上書きするかどうか
    save_to_drive : bool
        Googleドライブに保存するかどうか

    Returns:
    --------
    dict
        データセットパスの辞書
    """
    if datasets is None:
        datasets = [
            "numerai_training_data.parquet",
            "numerai_tournament_data.parquet",
            "numerai_validation_data.parquet",
            "example_predictions.parquet",
            "features.json"
        ]

    # ファイル名の短縮マッピング
    filename_map = {
        "numerai_training_data.parquet": "train.parquet",
        "numerai_tournament_data.parquet": "live.parquet",
        "numerai_validation_data.parquet": "validation.parquet",
        "example_predictions.parquet": "example_preds.parquet",
        "features.json": "features.json"
    }

    dataset_paths = {}
    napi = config.get('napi')

    if napi is None:
        print("有効なNumerai APIインスタンスがありません")
        return dataset_paths

    for dataset in datasets:
        short_name = filename_map.get(dataset, dataset)
        local_path = f"/content/{short_name}"

        # 最初にローカルに保存
        if not os.path.exists(local_path) or force_download:
            print(f"{dataset}をダウンロード中...")
            try:
                napi.download_dataset(dataset, local_path)
                print(f"  保存先: {local_path}")
            except Exception as e:
                print(f"  ダウンロードエラー: {e}")
                continue
        else:
            print(f"{short_name}は既にダウンロードされています")

        dataset_paths[short_name] = local_path

        # Googleドライブにコピー（オプション）
        if save_to_drive and 'drive_path' in config:
            drive_path = f"{config['drive_path']}/{short_name}"
            try:
                if not os.path.exists(drive_path) or force_download:
                    print(f"  Googleドライブにコピー中: {drive_path}")
                    !cp "{local_path}" "{drive_path}"
            except Exception as e:
                print(f"  ドライブへのコピーエラー: {e}")

    # features.jsonの読み込み
    if 'features.json' in dataset_paths:
        try:
            with open(dataset_paths['features.json'], 'r') as f:
                config['features_info'] = json.load(f)
            print("features.jsonを読み込みました")
        except Exception as e:
            print(f"features.json読み込みエラー: {e}")

    return dataset_paths

def load_numerai_data(dataset_paths, config=None):
    """
    ダウンロードしたNumeraiデータを読み込む関数

    Parameters:
    -----------
    dataset_paths : dict
        ダウンロード関数から返されるデータセットパスの辞書
    config : dict, optional
        設定情報

    Returns:
    --------
    dict
        データフレームの辞書
    """
    data = {}

    # 主要データセットの読み込み
    datasets_to_load = {
        'train.parquet': 'train_df',
        'live.parquet': 'tournament_df',
        'validation.parquet': 'validation_df',
        'example_preds.parquet': 'example_preds'
    }

    for file_key, df_key in datasets_to_load.items():
        if file_key in dataset_paths:
            try:
                print(f"{file_key}を読み込み中...")
                data[df_key] = pd.read_parquet(dataset_paths[file_key])
                print(f"  形状: {data[df_key].shape}")
            except Exception as e:
                print(f"  読み込みエラー: {e}")

    # 特徴量の検出
    if 'train_df' in data:
        data['feature_cols'] = [col for col in data['train_df'].columns
                               if col.startswith('feature')]
        print(f"検出された特徴量数: {len(data['feature_cols'])}")

    # 設定情報に特徴量グループが含まれている場合はそれを追加
    if config and 'features_info' in config:
        data['features_info'] = config['features_info']
        if 'feature_sets' in config['features_info']:
            print("特徴量セット情報を読み込みました")

    return data

In [5]:
# 通常のダウンサンプリング（現在の方法）
train = train[train["era"].isin(train["era"].unique()[::4])]

# より洗練されたサンプリング戦略
def smart_sampling(df, era_col="era", target_col=MAIN_TARGET, sample_ratio=0.25):
    """より賢いサンプリング方法：時間的連続性と分布の保持を重視"""
    unique_eras = df[era_col].unique()

    # 最新エラを優先サンプリング（より最近のデータは価値が高い）
    recent_eras = unique_eras[-int(len(unique_eras) * 0.3):]  # 最新30%
    older_eras = unique_eras[:-int(len(unique_eras) * 0.3)]   # 残り70%

    # 最新エラは高い比率で、古いエラは低い比率でサンプリング
    sampled_recent = recent_eras[::max(1, int(1/sample_ratio * 0.5))]  # 最新エラは2倍の密度
    sampled_older = older_eras[::max(1, int(1/sample_ratio * 1.5))]    # 古いエラは2/3の密度

    sampled_eras = np.concatenate([sampled_older, sampled_recent])
    return df[df[era_col].isin(sampled_eras)]

train = smart_sampling(train, sample_ratio=0.25)  # 全体の25%を使用

In [6]:
def ultra_efficient_feature_engineering(df, feature_cols, era_col="era", features_info=None):
    """計算効率を極限まで高めた特徴エンジニアリング"""
    print("特徴エンジニアリングを開始...")
    result_df = df.copy()

    # 実際にデータフレームに存在する特徴量のみをフィルタリング
    valid_feature_cols = [col for col in feature_cols if col in df.columns]
    print(f"有効な特徴量: {len(valid_feature_cols)}/{len(feature_cols)}")

    if len(valid_feature_cols) == 0:
        print("警告: 有効な特徴量が見つかりません。データを確認してください。")
        return result_df

    # 1. メモリ最適化: float64からfloat32へ変換
    for col in valid_feature_cols:
        if result_df[col].dtype == 'float64':
            result_df[col] = result_df[col].astype('float32')

    # 2. 最も効果的な変換だけを適用（ランク変換と正規化）
    for era in df[era_col].unique():
        era_data = df[df[era_col] == era]

        # 特徴重要度情報があれば、それを使用
        if features_info and 'feature_stats' in features_info:
            # 存在する特徴だけをフィルタリング
            valid_stats = {f: features_info['feature_stats'][f]
                          for f in valid_feature_cols
                          if f in features_info['feature_stats']}

            if valid_stats:
                top_features = sorted(
                    [(f, valid_stats[f].get('corr', 0)) for f in valid_stats],
                    key=lambda x: abs(x[1]),
                    reverse=True
                )[:min(50, len(valid_stats))]  # 上位50特徴または全特徴
                selected_features = [f for f, _ in top_features]
            else:
                selected_features = valid_feature_cols[:min(50, len(valid_feature_cols))]
        else:
            selected_features = valid_feature_cols[:min(50, len(valid_feature_cols))]

        # 最も効果的なランク変換だけを適用
        for col in selected_features[:min(15, len(selected_features))]:  # 最大15特徴
            result_df.loc[result_df[era_col] == era, f'{col}_rank'] = \
                era_data[col].rank(pct=True).astype('float32')

        # 最も効果的なエラ内正規化だけを適用
        for col in selected_features[:min(20, len(selected_features))]:  # 最大20特徴
            mean = era_data[col].mean()
            std = era_data[col].std()
            if std > 0:
                result_df.loc[result_df[era_col] == era, f'{col}_norm'] = \
                    ((result_df.loc[result_df[era_col] == era, col] - mean) / std).astype('float32')

    # 3. 最小限のクロス特徴（相互作用）
    top_pairs = [
        ('intelligence', 'strength'),
        ('wisdom', 'charisma')
    ]

    # グループ情報がある場合のみ実行
    if features_info and 'feature_sets' in features_info:
        for group1, group2 in top_pairs:
            if group1 in features_info['feature_sets'] and group2 in features_info['feature_sets']:
                # 実際にデータフレームに存在する特徴のみをフィルタリング
                g1_features = [f for f in features_info['feature_sets'].get(group1, [])[:3]
                              if f in df.columns]  # 最大3特徴
                g2_features = [f for f in features_info['feature_sets'].get(group2, [])[:3]
                              if f in df.columns]  # 最大3特徴

                for f1 in g1_features:
                    for f2 in g2_features:
                        result_df[f'{f1}_mul_{f2}'] = (result_df[f1] * result_df[f2]).astype('float32')

    # 4. シンプルなグループ統計（最も重要なグループのみ）
    if features_info and 'feature_sets' in features_info:
        for group in ['intelligence', 'strength']:  # 最重要グループのみ
            if group in features_info['feature_sets']:
                # 実際にデータフレームに存在する特徴のみをフィルタリング
                group_features = [f for f in features_info['feature_sets'][group][:20]
                                 if f in df.columns]  # 最大20特徴
                if len(group_features) >= 5:
                    result_df[f'mean_{group}'] = result_df[group_features].mean(axis=1).astype('float32')

    # キャスト後のメモリ使用量
    num_bytes = result_df.memory_usage(deep=True).sum()
    print(f"メモリ使用量: {num_bytes / 1e6:.2f} MB")

    print("特徴エンジニアリング完了")
    return result_df

In [7]:
def ultra_fast_feature_selection(df, target_col, feature_cols, era_col="era"):
    """超高速な特徴選択（計算コスト最小化）"""
    from sklearn.feature_selection import SelectFromModel
    from sklearn.linear_model import LassoCV  # LightGBMより高速

    # 特徴のフィルタリング（より計算効率の高い方法）
    all_features = [col for col in df.columns
                   if col.startswith('feature') or
                   '_rank' in col or '_norm' in col or
                   '_mul_' in col or 'pca_' in col or
                   col.startswith('mean_')]

    # サンプリングによるメモリ効率化
    sample_size = min(50000, len(df))
    sample_idx = np.random.choice(len(df), sample_size, replace=False)
    sample_df = df.iloc[sample_idx]

    # 高速な特徴選択モデル
    selector = LassoCV(cv=3, max_iter=100, tol=0.01)
    selector.fit(sample_df[all_features], sample_df[target_col])

    # 重要な特徴の選択
    importance = np.abs(selector.coef_)
    feature_importance = pd.Series(importance, index=all_features)
    feature_importance = feature_importance.sort_values(ascending=False)

    # 上位特徴のみを選択
    top_features = feature_importance.index[:200].tolist()  # 200特徴に制限

    return top_features

def train_minimal_models(train_df, features, target_col, era_col="era"):
    """リソース制約下での最小限のモデルトレーニング"""
    from sklearn.linear_model import Ridge
    from lightgbm import LGBMRegressor

    # サンプリングによるメモリ効率化
    sample_size = min(50000, len(train_df))
    sample_idx = np.random.choice(len(train_df), sample_size, replace=False)
    sample_df = train_df.iloc[sample_idx]

    # 特徴とターゲットの取得
    X = sample_df[features]
    y = sample_df[target_col]

    # 1. Ridge回帰（計算効率が非常に高い）
    ridge_model = Ridge(alpha=1.0, random_state=42)
    ridge_model.fit(X, y)

    # 2. 軽量LightGBM（パラメータを最小化）
    lgbm_model = LGBMRegressor(
        n_estimators=500,  # 1000から半減
        learning_rate=0.02,  # 学習率を上げて反復回数削減
        num_leaves=15,     # 葉の数を減らす
        max_depth=4,       # 深さを制限
        colsample_bytree=0.7,
        subsample=0.7,
        verbose=-1,
        random_state=42
    )

    lgbm_model.fit(X, y)

    # モデルをまとめる
    models = {
        'ridge': ridge_model,
        'lgbm': lgbm_model,
        'weights': {'ridge': 0.3, 'lgbm': 0.7}  # 経験的に良い比率
    }

    return models

In [8]:
def streamlined_prediction_optimization(predictions_df, feature_df, era_col="era",
                                       example_preds=None, feature_groups=None):
    """リソース効率を最大化した予測最適化"""
    result_df = predictions_df.copy()

    # 1. 効率的なエラ内正規化
    for era in result_df[era_col].unique():
        era_mask = result_df[era_col] == era
        if era_mask.sum() >= 2:
            era_preds = result_df.loc[era_mask, 'prediction']
            mean = np.mean(era_preds)
            std = np.std(era_preds)
            if std > 0:
                result_df.loc[era_mask, 'prediction'] = (era_preds - mean) / std

    # 2. 最小限の特徴中立化（最も重要な特徴グループのみ）
    if feature_df is not None and feature_groups:
        # 最も重要なグループだけを中立化
        neutralize_groups = {
            'intelligence': 0.5,  # 最重要グループ
            'strength': 0.5       # 最重要グループ
        }

        for group_name, proportion in neutralize_groups.items():
            if group_name in feature_groups and len(feature_groups[group_name]) > 0:
                print(f"  {group_name}グループを中立化中...")
                neutral_cols = feature_groups[group_name][:50]  # 上位50特徴に制限

                # 一度にすべてのエラを処理（バッチ処理をスキップしてコード簡略化）
                from sklearn.linear_model import Ridge  # LinearRegressionより安定

                for era in result_df[era_col].unique():
                    era_mask = (result_df[era_col] == era)
                    if era_mask.sum() < 10:
                        continue

                    era_indices = result_df.loc[era_mask].index
                    era_features = feature_df.loc[era_indices, neutral_cols]
                    era_predictions = result_df.loc[era_mask, 'prediction'].values.reshape(-1, 1)

                    try:
                        # Ridge回帰で中立化（より安定）
                        model = Ridge(alpha=0.01, fit_intercept=False).fit(era_features, era_predictions)
                        neutralized = era_predictions - proportion * model.predict(era_features).reshape(-1, 1)
                        result_df.loc[era_indices, 'prediction'] = neutralized.flatten()
                    except:
                        continue

    # 3. 簡略化したMMC最適化
    if example_preds is not None:
        try:
            if 'prediction' in example_preds.columns:
                merged = pd.merge(
                    result_df[['id', 'prediction']],
                    example_preds[['id', 'prediction']],
                    on='id',
                    suffixes=('', '_example')
                )

                # 相関が高い場合のみ差別化を適用（計算を節約）
                corr = np.corrcoef(merged['prediction'], merged['prediction_example'])[0, 1]
                print(f"  メタモデルとの相関: {corr:.4f}")

                if corr > 0.75:  # 高い相関の場合のみ
                    # 簡易的な差別化（より効率的）
                    diff_strength = 0.2  # 固定値を使用して計算を簡略化

                    # 直接計算（メモリマッピングを回避）
                    merged['prediction_mmc'] = merged['prediction'] + diff_strength * (
                        merged['prediction'] - merged['prediction_example'])

                    # マッピングを更新
                    prediction_map = dict(zip(merged['id'], merged['prediction_mmc']))
                    result_df['prediction'] = result_df['id'].map(prediction_map).fillna(result_df['prediction'])
        except Exception as e:
            print(f"MMC最適化エラー: {e}")

    # 4. 最終クリッピング
    result_df['prediction'] = result_df['prediction'].clip(-3, 3)

    return result_df

In [9]:
def numerai_lightweight_pipeline():
    """Colabの無料枠内で実行可能な軽量版ワークフロー"""
    # データ読み込み（既存のコードを使用）
    feature_metadata = json.load(open(f"{DATA_VERSION}/features.json"))
    feature_cols = feature_metadata["feature_sets"]["medium"]
    target_cols = feature_metadata["targets"]

    # トレーニングデータの読み込みとスマートサンプリング
    train = pd.read_parquet(
        f"{DATA_VERSION}/train.parquet",
        columns=["era"] + feature_cols + target_cols
    )
    train = smart_sampling(train, era_col="era", target_col=MAIN_TARGET, sample_ratio=0.25)

    # 超効率的な特徴エンジニアリング
    print("超効率的な特徴エンジニアリングを実行中...")
    train_engineered = ultra_efficient_feature_engineering(
        train, feature_cols, era_col="era", features_info=feature_metadata
    )

    # 高速特徴選択
    print("高速特徴選択を実行中...")
    selected_features = ultra_fast_feature_selection(
        train_engineered, MAIN_TARGET, feature_cols, era_col="era"
    )

    # 軽量モデルトレーニング
    print("軽量モデルトレーニングを実行中...")
    models = train_minimal_models(
        train_engineered, selected_features, MAIN_TARGET, era_col="era"
    )

    # トーナメントデータ処理
    print("トーナメントデータを処理中...")
    tournament = pd.read_parquet(
        f"{DATA_VERSION}/live.parquet",
        columns=["era", "id"] + feature_cols
    )

    # トーナメントデータに同じ特徴エンジニアリングを適用
    tournament_engineered = ultra_efficient_feature_engineering(
        tournament, feature_cols, era_col="era", features_info=feature_metadata
    )

    # 予測生成
    print("予測を生成中...")
    # 各モデルの予測を計算
    model_preds = {}
    for name, model in models.items():
        if name != 'weights':
            model_preds[name] = model.predict(tournament_engineered[selected_features])

    # 重み付けアンサンブル
    weights = models.get('weights', {})
    ensemble_pred = np.zeros(len(tournament_engineered))
    for name, preds in model_preds.items():
        if name in weights:
            ensemble_pred += weights[name] * preds
        else:
            ensemble_pred += preds / len(model_preds)

    # 予測フォーマット
    predictions_df = pd.DataFrame({
        'id': tournament_engineered["id"],
        'era': tournament_engineered["era"],
        'prediction': ensemble_pred
    })

    # 最適化
    print("予測を最適化中...")
    # 例示予測の読み込み
    example_preds = pd.read_parquet(f"{DATA_VERSION}/live_example_preds.parquet")

    # 特徴グループ（メタデータから）
    feature_groups = feature_metadata.get("feature_sets", {})

    # 効率的な予測最適化
    optimized_preds = streamlined_prediction_optimization(
        predictions_df,
        tournament_engineered,
        era_col="era",
        example_preds=example_preds,
        feature_groups=feature_groups
    )

    # 結果を保存
    final_predictions = optimized_preds[['id', 'prediction']].copy()
    submission_path = f"numerai_lightweight_submission.csv"
    final_predictions.to_csv(submission_path, index=False)

    print(f"軽量パイプライン完了! 予測保存先: {submission_path}")

    return final_predictions, models

In [10]:
def nan_safe_feature_selection(df, target_col, feature_cols, era_col="era"):
    """NaN値に対応した特徴選択関数"""
    from sklearn.linear_model import LassoCV
    from sklearn.impute import SimpleImputer
    import numpy as np

    print("NaN値に対応した特徴選択を実行中...")

    # 拡張特徴を検出
    all_features = [col for col in df.columns
                   if col.startswith('feature') or
                   '_rank' in col or '_norm' in col or
                   '_mul_' in col or 'pca_' in col or
                   col.startswith('mean_')]

    # 実際に存在する特徴のみ使用
    valid_features = [f for f in all_features if f in df.columns]
    print(f"有効な特徴量: {len(valid_features)}")

    if len(valid_features) == 0:
        print("警告: 有効な特徴量が見つかりません。基本特徴量を返します。")
        return feature_cols[:min(200, len(feature_cols))]

    # サンプリングでメモリ効率化
    sample_size = min(50000, len(df))
    sample_idx = np.random.choice(len(df), sample_size, replace=False)
    sample_df = df.iloc[sample_idx]

    # 特徴行列とターゲット
    X = sample_df[valid_features]
    y = sample_df[target_col]

    # NaN値のチェック
    nan_count = X.isna().sum().sum()
    print(f"特徴行列中のNaN値の総数: {nan_count}")

    # NaN値を平均値で補完
    print("欠損値を平均値で補完しています...")
    imputer = SimpleImputer(strategy='mean')
    X_imputed = imputer.fit_transform(X)

    # LassoCVによる特徴選択
    print("LassoCVで特徴選択を実行中...")
    selector = LassoCV(cv=3, max_iter=100, tol=0.01, random_state=42)

    try:
        selector.fit(X_imputed, y)

        # 特徴重要度
        importance = np.abs(selector.coef_)
        feature_importance = pd.Series(importance, index=valid_features)
        feature_importance = feature_importance.sort_values(ascending=False)

        # 重要な特徴のみを選択
        selected_features = feature_importance.index[:min(200, len(feature_importance))].tolist()
        print(f"選択された特徴数: {len(selected_features)}")

        return selected_features

    except Exception as e:
        print(f"モデル訓練中にエラー発生: {e}")
        print("代替手法として相関ベースの特徴選択を使用します...")
        return correlation_based_selection(sample_df, valid_features, target_col, max_features=200)

def correlation_based_selection(df, features, target_col, max_features=200):
    """相関係数に基づく特徴選択（LassoCVのフォールバック）"""
    correlations = {}

    for feature in features:
        # NaN値を除外して相関を計算
        mask = ~(df[feature].isna() | df[target_col].isna())
        if mask.sum() > 10:  # 十分なデータがある場合
            try:
                corr = np.abs(np.corrcoef(df.loc[mask, feature], df.loc[mask, target_col])[0, 1])
                if not np.isnan(corr):
                    correlations[feature] = corr
            except:
                pass

    # 相関でソート
    sorted_features = sorted(correlations.items(), key=lambda x: x[1], reverse=True)
    selected = [f for f, _ in sorted_features[:max_features]]

    print(f"相関ベースの選択で{len(selected)}個の特徴を選択")
    return selected

In [11]:
def robust_minimal_models(train_df, features, target_col, era_col="era"):
    """NaN値に対応した堅牢な軽量モデルトレーニング"""
    from sklearn.linear_model import Ridge
    from sklearn.impute import SimpleImputer
    from lightgbm import LGBMRegressor

    print("NaN値に対応したモデルトレーニングを実行中...")

    # サンプリングでメモリ効率化
    sample_size = min(50000, len(train_df))
    sample_idx = np.random.choice(len(train_df), sample_size, replace=False)
    sample_df = train_df.iloc[sample_idx]

    # 特徴行列とターゲット
    X = sample_df[features]
    y = sample_df[target_col]

    # NaN値のチェック
    nan_count = X.isna().sum().sum()
    print(f"トレーニングデータのNaN値の総数: {nan_count}")

    # NaN値を平均値で補完
    print("欠損値を平均値で補完しています...")
    imputer = SimpleImputer(strategy='mean')
    X_imputed = imputer.fit_transform(X)

    # 保存用にimputer保持

    # 1. Ridge回帰 (安定性重視)
    print("Ridge回帰モデルをトレーニング中...")
    ridge_model = Ridge(alpha=1.0, random_state=42)
    ridge_model.fit(X_imputed, y)

    # 2. 軽量版LightGBM
    print("軽量版LightGBMモデルをトレーニング中...")
    lgbm_model = LGBMRegressor(
        n_estimators=500,
        learning_rate=0.02,
        num_leaves=15,
        max_depth=4,
        colsample_bytree=0.7,
        subsample=0.7,
        verbose=-1,
        random_state=42
    )

    try:
        lgbm_model.fit(X_imputed, y)
        lgbm_trained = True
    except Exception as e:
        print(f"LightGBMトレーニングエラー: {e}")
        print("LightGBMをスキップし、Ridgeのみを使用します")
        lgbm_trained = False

    # モデルとimputer
    models = {
        'ridge': ridge_model,
        'imputer': imputer
    }

    if lgbm_trained:
        models['lgbm'] = lgbm_model
        models['weights'] = {'ridge': 0.3, 'lgbm': 0.7}  # 経験的に良い比率
    else:
        models['weights'] = {'ridge': 1.0}  # Ridgeのみ

    return models

In [12]:
def nan_safe_prediction(models, tournament_df, features, era_col="era"):
    """NaN値に対応した予測生成"""
    print("NaN値に対応した予測を生成中...")

    # 特徴行列抽出
    X_test = tournament_df[features]

    # NaN値のチェック
    nan_count = X_test.isna().sum().sum()
    print(f"テストデータのNaN値の総数: {nan_count}")

    # 保存したimputerで補完
    imputer = models['imputer']
    X_test_imputed = imputer.transform(X_test)

    # 各モデルの予測
    model_preds = {}
    weights = models['weights']

    for name, model in models.items():
        if name not in ['imputer', 'weights']:
            model_preds[name] = model.predict(X_test_imputed)

    # 重み付きアンサンブル
    ensemble_pred = np.zeros(len(tournament_df))

    for name, preds in model_preds.items():
        if name in weights:
            ensemble_pred += weights[name] * preds

    # 予測フォーマット
    predictions_df = pd.DataFrame({
        'id': tournament_df["id"],
        'era': tournament_df[era_col],
        'prediction': ensemble_pred
    })

    return predictions_df

def simplified_optimization(predictions_df, feature_df=None, era_col="era",
                           example_preds=None, feature_groups=None):
    """シンプル化した予測最適化"""
    print("軽量化した予測最適化を実行中...")
    result_df = predictions_df.copy()

    # 1. エラ内正規化（必須）
    for era in result_df[era_col].unique():
        era_mask = result_df[era_col] == era
        if era_mask.sum() >= 2:
            era_preds = result_df.loc[era_mask, 'prediction']
            mean = np.mean(era_preds)
            std = np.std(era_preds)
            if std > 0:
                result_df.loc[era_mask, 'prediction'] = (era_preds - mean) / std

    # 2. MMC最適化（メタモデルとの差別化）- 最も効果的な部分
    if example_preds is not None:
        try:
            merged = pd.merge(
                result_df[['id', 'prediction']],
                example_preds[['id', 'prediction']],
                on='id',
                suffixes=('', '_example')
            )

            # 相関計算
            corr = np.corrcoef(merged['prediction'], merged['prediction_example'])[0, 1]
            print(f"メタモデルとの相関: {corr:.4f}")

            if corr > 0.7:  # 高相関の場合のみ差別化
                # 簡易的な差別化
                diff_strength = 0.2
                merged['prediction_mmc'] = merged['prediction'] + diff_strength * (
                    merged['prediction'] - merged['prediction_example'])

                # IDによるマッピング更新
                prediction_map = dict(zip(merged['id'], merged['prediction_mmc']))
                result_df['prediction'] = result_df['id'].map(prediction_map).fillna(result_df['prediction'])
        except Exception as e:
            print(f"MMC最適化エラー: {e}")

    # 3. 最終クリッピング
    result_df['prediction'] = result_df['prediction'].clip(-3, 3)

    return result_df

In [13]:
def numerai_minimal_pipeline():
    """超軽量Numeraiパイプライン - NaN対応版とパス修正版"""
    print("超軽量Numeraiパイプライン（NaN対応版）を開始...")

    # 1. まずファイルの存在確認
    print("必要なファイルの存在を確認中...")
    required_files = [
        "train.parquet",
        "features.json",
        "live.parquet",
        "live_example_preds.parquet"
    ]

    # ファイルが存在するか確認し、存在しない場合はダウンロード
    for file in required_files:
        # まず直接ファイル名で確認
        if os.path.exists(file):
            print(f"  ✓ {file} が存在します")
        # DATA_VERSIONディレクトリ内を確認
        elif os.path.exists(f"{DATA_VERSION}/{file}"):
            print(f"  ✓ {DATA_VERSION}/{file} が存在します")
        else:
            print(f"  ✗ {file} が見つかりません。ダウンロードを試みます...")
            try:
                napi = NumerAPI()
                if DATA_VERSION == "v5.0":
                    # v5.0の場合はパスを入れる
                    napi.download_dataset(f"{DATA_VERSION}/{file}")
                else:
                    # その他の場合は直接ファイル名
                    napi.download_dataset(file)
                print(f"    ダウンロード完了: {file}")
            except Exception as e:
                print(f"    ダウンロードエラー: {e}")
                return None, None

    # 2. 適切なパスでファイルを読み込む
    print("データ読み込みとサンプリングを開始...")

    # ファイルパスを適切に判断する関数
    def get_file_path(filename):
        if os.path.exists(filename):
            return filename
        elif os.path.exists(f"{DATA_VERSION}/{filename}"):
            return f"{DATA_VERSION}/{filename}"
        else:
            raise FileNotFoundError(f"ファイルが見つかりません: {filename}")

    # features.jsonの読み込み
    feature_metadata = json.load(open(get_file_path("features.json")))
    feature_cols = feature_metadata["feature_sets"]["medium"]
    target_cols = feature_metadata["targets"]

    # トレーニングデータの読み込み
    train = pd.read_parquet(
        get_file_path("train.parquet"),
        columns=["era"] + feature_cols + [MAIN_TARGET]
    )

    # スマートサンプリング
    train_sampled = train[train["era"].isin(train["era"].unique()[::4])]
    print(f"サンプリング後のデータサイズ: {len(train_sampled)}")

    # 3. 最小限の特徴エンジニアリング (以下は同じ)
    print("最小限の特徴エンジニアリングを実行中...")
    train_engineered = simple_feature_engineering(train_sampled, era_col="era")

    # 4. NaN対応特徴選択
    print("NaN対応特徴選択を実行中...")
    selected_features = nan_safe_feature_selection(
        train_engineered, MAIN_TARGET, feature_cols, era_col="era"
    )

    # 5. 堅牢なモデルトレーニング
    print("堅牢なモデルトレーニングを実行中...")
    models = robust_minimal_models(
        train_engineered, selected_features, MAIN_TARGET, era_col="era"
    )

    # 6. トーナメントデータ処理 - パスを修正
    print("トーナメントデータを処理中...")
    tournament = pd.read_parquet(
        get_file_path("live.parquet"),
        columns=["era", "id"] + feature_cols
    )

    # 同じ特徴エンジニアリングを適用
    tournament_engineered = simple_feature_engineering(
        tournament, era_col="era"
    )

    # 7. 予測生成
    print("予測を生成中...")
    predictions_df = nan_safe_prediction(
        models, tournament_engineered, selected_features, era_col="era"
    )

    # 8. 予測最適化 - パスを修正
    print("予測を最適化中...")
    example_preds = pd.read_parquet(get_file_path("live_example_preds.parquet"))

    optimized_preds = simplified_optimization(
        predictions_df,
        era_col="era",
        example_preds=example_preds
    )

    # 9. 結果保存
    final_predictions = optimized_preds[['id', 'prediction']].copy()
    submission_path = f"numerai_minimal_submission.csv"
    final_predictions.to_csv(submission_path, index=False)

    print(f"軽量パイプライン完了! 予測保存先: {submission_path}")

    return final_predictions, models

In [14]:
def simple_feature_engineering(df, era_col="era"):
    """超簡易版の特徴エンジニアリング - NaN値に強い"""
    print("簡易特徴エンジニアリングを開始...")
    result_df = df.copy()

    # 特徴量を検出
    feature_cols = [col for col in df.columns if col.startswith('feature')]
    print(f"検出された特徴量: {len(feature_cols)}")

    if len(feature_cols) == 0:
        print("警告: 特徴量が見つかりません")
        return result_df

    # 最小限の変換のみ実行
    # 1. 上位20特徴に対してランク変換（NaN値に強い）
    top_features = feature_cols[:min(20, len(feature_cols))]

    for col in top_features:
        # NaN値を含む可能性があるのでtry-exceptで囲む
        try:
            result_df[f'{col}_rank'] = result_df.groupby(era_col)[col].rank(pct=True)
        except:
            print(f"Warning: {col}のランク変換に失敗")

    # 2. 最小限のクロス特徴（上位5特徴間の積）
    for i, feat1 in enumerate(top_features[:5]):
        for feat2 in top_features[i+1:min(i+3, len(top_features))]:
            try:
                result_df[f'{feat1}_mul_{feat2}'] = result_df[feat1] * result_df[feat2]
            except:
                pass

    print("簡易特徴エンジニアリング完了")
    return result_df

In [15]:
def numerai_minimal_pipeline():
    """超軽量Numeraiパイプライン - NaN対応版とパス修正版"""
    print("超軽量Numeraiパイプライン（NaN対応版）を開始...")

    # 1. まずファイルの存在確認
    print("必要なファイルの存在を確認中...")
    required_files = [
        "train.parquet",
        "features.json",
        "live.parquet",
        "live_example_preds.parquet"
    ]

    # ファイルが存在するか確認し、存在しない場合はダウンロード
    for file in required_files:
        # まず直接ファイル名で確認
        if os.path.exists(file):
            print(f"  ✓ {file} が存在します")
        # DATA_VERSIONディレクトリ内を確認
        elif os.path.exists(f"{DATA_VERSION}/{file}"):
            print(f"  ✓ {DATA_VERSION}/{file} が存在します")
        else:
            print(f"  ✗ {file} が見つかりません。ダウンロードを試みます...")
            try:
                napi = NumerAPI()
                if DATA_VERSION == "v5.0":
                    # v5.0の場合はパスを入れる
                    napi.download_dataset(f"{DATA_VERSION}/{file}")
                else:
                    # その他の場合は直接ファイル名
                    napi.download_dataset(file)
                print(f"    ダウンロード完了: {file}")
            except Exception as e:
                print(f"    ダウンロードエラー: {e}")
                return None, None

    # 2. 適切なパスでファイルを読み込む
    print("データ読み込みとサンプリングを開始...")

    # ファイルパスを適切に判断する関数
    def get_file_path(filename):
        if os.path.exists(filename):
            return filename
        elif os.path.exists(f"{DATA_VERSION}/{filename}"):
            return f"{DATA_VERSION}/{filename}"
        else:
            raise FileNotFoundError(f"ファイルが見つかりません: {filename}")

    # features.jsonの読み込み
    feature_metadata = json.load(open(get_file_path("features.json")))
    feature_cols = feature_metadata["feature_sets"]["medium"]
    target_cols = feature_metadata["targets"]

    # トレーニングデータの読み込み
    train = pd.read_parquet(
        get_file_path("train.parquet"),
        columns=["era"] + feature_cols + [MAIN_TARGET]
    )

    # スマートサンプリング
    train_sampled = train[train["era"].isin(train["era"].unique()[::4])]
    print(f"サンプリング後のデータサイズ: {len(train_sampled)}")

    # 3. 最小限の特徴エンジニアリング (以下は同じ)
    print("最小限の特徴エンジニアリングを実行中...")
    train_engineered = simple_feature_engineering(train_sampled, era_col="era")

    # 4. NaN対応特徴選択
    print("NaN対応特徴選択を実行中...")
    selected_features = nan_safe_feature_selection(
        train_engineered, MAIN_TARGET, feature_cols, era_col="era"
    )

    # 5. 堅牢なモデルトレーニング
    print("堅牢なモデルトレーニングを実行中...")
    models = robust_minimal_models(
        train_engineered, selected_features, MAIN_TARGET, era_col="era"
    )

    # 6. トーナメントデータ処理 - パスを修正
    print("トーナメントデータを処理中...")
    tournament = pd.read_parquet(
        get_file_path("live.parquet"),
        columns=["era", "id"] + feature_cols
    )

    # 同じ特徴エンジニアリングを適用
    tournament_engineered = simple_feature_engineering(
        tournament, era_col="era"
    )

    # 7. 予測生成
    print("予測を生成中...")
    predictions_df = nan_safe_prediction(
        models, tournament_engineered, selected_features, era_col="era"
    )

    # 8. 予測最適化 - パスを修正
    print("予測を最適化中...")
    example_preds = pd.read_parquet(get_file_path("live_example_preds.parquet"))

    optimized_preds = simplified_optimization(
        predictions_df,
        era_col="era",
        example_preds=example_preds
    )

    # 9. 結果保存
    final_predictions = optimized_preds[['id', 'prediction']].copy()
    submission_path = f"numerai_minimal_submission.csv"
    final_predictions.to_csv(submission_path, index=False)

    print(f"軽量パイプライン完了! 予測保存先: {submission_path}")

    return final_predictions, models

In [ ]:
# osモジュールをインポート
import os

# 全ての関数を定義した後、最後にこれを実行
if __name__ == "__main__":
    print("パス修正版の超軽量Numeraiパイプラインを実行します...")
    final_predictions, models = numerai_minimal_pipeline()

パス修正版の超軽量Numeraiパイプラインを実行します...
超軽量Numeraiパイプライン（NaN対応版）を開始...
必要なファイルの存在を確認中...
  ✓ v5.0/train.parquet が存在します
  ✓ v5.0/features.json が存在します
  ✗ live.parquet が見つかりません。ダウンロードを試みます...


v5.0/live.parquet: 8.03MB [00:00, 19.2MB/s]                            


    ダウンロード完了: live.parquet
  ✓ v5.0/live_example_preds.parquet が存在します
データ読み込みとサンプリングを開始...
サンプリング後のデータサイズ: 688184
最小限の特徴エンジニアリングを実行中...
簡易特徴エンジニアリングを開始...
検出された特徴量: 705
簡易特徴エンジニアリング完了
NaN対応特徴選択を実行中...
NaN値に対応した特徴選択を実行中...
有効な特徴量: 735
特徴行列中のNaN値の総数: 0
欠損値を平均値で補完しています...
LassoCVで特徴選択を実行中...
